## Kaggle Start Book in Python

(PythonではじめるKaggleスタートブック)

Ishihara, Shotaro; Murata, Hideki Practical Data Science Series: Kaggle Start Book in Python (KS Information Science Specialized Book) . Kodansha. Kindle edition.

石原祥太郎; 村田秀樹. 実践Data Scienceシリーズ PythonではじめるKaggleスタートブック (ＫＳ情報科学専門書) . 講談社.


# 2.5 勾配ブースティングが最強？！ いろいろな機械学習アルゴリズムを使ってみよう

# (section 2.5 Is gradient boosting the best? Let's try different machine learning algorithms!)

original NoteBook :

https://www.kaggle.com/code/sishihara/python-kaggle-start-book-ch02-05

In [ ]:
import numpy as np
import pandas as pd


train = pd.read_csv('../input/titanic/train.csv')
test = pd.read_csv('../input/titanic/test.csv')
gender_submission = pd.read_csv('../input/titanic/gender_submission.csv')

data = pd.concat([train, test], sort=False)

data['Sex'].replace(['male', 'female'], [0, 1], inplace=True)
data['Embarked'].fillna(('S'), inplace=True)
data['Embarked'] = data['Embarked'].map({'S': 0, 'C': 1, 'Q': 2}).astype(int)
data['Fare'].fillna(np.mean(data['Fare']), inplace=True)
data['Age'].fillna(data['Age'].median(), inplace=True)
data['FamilySize'] = data['Parch'] + data['SibSp'] + 1
data['IsAlone'] = 0
data.loc[data['FamilySize'] == 1, 'IsAlone'] = 1

In [ ]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamilySize,IsAlone
0,1,0.0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,0,2,0
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,1,2,0
2,3,1.0,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,0,1,1
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,0,2,0
4,5,0.0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,NaN,0,1,1


In [ ]:
delete_columns = ['Name', 'PassengerId', 'Ticket', 'Cabin']
data.drop(delete_columns, axis=1, inplace=True)

train = data[:len(train)]
test = data[len(train):]

y_train = train['Survived']
X_train = train.drop('Survived', axis=1)
X_test = test.drop('Survived', axis=1)

In [ ]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,FamilySize,IsAlone
0,3,0,22.0,1,0,7.2500,0,2,0
1,1,1,38.0,1,0,71.2833,1,2,0
2,3,1,26.0,0,0,7.9250,0,1,1
3,1,1,35.0,1,0,53.1000,0,2,0
4,3,0,35.0,0,0,8.0500,0,1,1


# sklearn

In [ ]:
from sklearn.linear_model import LogisticRegression


clf = LogisticRegression(penalty='l2', solver='sag', random_state=0)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:
from sklearn.ensemble import RandomForestClassifier


clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)

In [ ]:
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [ ]:
y_pred[:10]

array([0., 1., 0., 0., 1., 0., 1., 0., 1., 0.])

In [ ]:
sub = pd.read_csv('../input/titanic/gender_submission.csv')

In [ ]:
sub['Survived'] = list(map(int, y_pred))
sub.to_csv('submission_randomforest.csv', index=False)
sub.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


# LightGBM

> By specifying y_ train in stratify, the ratio of the training dataset and the validation dataset after the split will be equal.

Ishihara, Shotaro; Murata, Hideki Kaggle Start Book in Python (KS Information Science Specialized Book) (p.252). Kodansha. Kindle edition.

In [ ]:
from sklearn.model_selection import train_test_split


X_train, X_valid, y_train, y_valid = \
    train_test_split(X_train, y_train, test_size=0.3,
                     random_state=0, stratify=y_train)

In [ ]:
categorical_features = ['Embarked', 'Pclass', 'Sex']

> Dataset is passed to the validation dataset as a reference to the training dataset lightgbm.

Ishihara, Shotaro; Murata, Hideki Kaggle Start Book in Python (KS Information Science Specialized Book) (p.253). Kodansha. Kindle edition.

In [ ]:
import lightgbm as lgb


lgb_train = lgb.Dataset(X_train, y_train,
                        categorical_feature=categorical_features)
lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train,
                       categorical_feature=categorical_features)

params = {
    'objective': 'binary'
}

model = lgb.train(params, lgb_train,
                  valid_sets=[lgb_train, lgb_eval],
                  num_boost_round=1000,
                  callbacks=[lgb.early_stopping(stopping_rounds=10,
                                                verbose=True),
                             lgb.log_evaluation(10)])

y_pred = model.predict(X_test, num_iteration=model.best_iteration)

/opt/conda/lib/python3.10/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/opt/conda/lib/python3.10/site-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/opt/conda/lib/python3.10/site-packages/lightgbm/basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[LightGBM] [Info] Number of positive: 239, number of negative: 384
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001120 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 205
[LightGBM] [Info] Number of data points in the train set: 623, number of used features: 9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.383628 -> initscore=-0.474179
[LightGBM] [Info] Start training from score -0.474179
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 10 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


In [ ]:
y_pred[:10]

array([0.0605672 , 0.53340112, 0.13712618, 0.07971355, 0.39226988,
       0.50073752, 0.71047286, 0.17751357, 0.70499098, 0.05328827])

>In this case, the pred value is a continuous value between 0 and 1.

>If y_ pred > 0.5, the value is converted to a discrete value y == 1, otherwise y == 0. Since “y_ pred > 0.5” yields a boolean value (true or false), it is converted to “1 or 0” using astype( int).

Ishihara, Shotaro; Murata, Hideki Kaggle Start Book in Python (KS Information Science Specialized Book) (p.253). Kodansha. Kindle edition.

In [ ]:
y_pred = (y_pred > 0.5).astype(int)
y_pred[:10]

array([0, 1, 0, 0, 0, 1, 1, 0, 1, 0])

In [ ]:
sub['Survived'] = y_pred
sub.to_csv('submission_lightgbm.csv', index=False)

sub.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,0


In [ ]:
sub.to_csv('submission.csv', index=False)